In [1]:
%run "/home/jan/Documentos/tcc/dependencias/conn_spark.py"

spark = cria_conn("gold")

your 131072x1 screen size is bogus. expect trouble
25/03/07 20:10:16 WARN Utils: Your hostname, DESKTOP-RE2QNUL resolves to a loopback address: 127.0.1.1; using 172.21.253.66 instead (on interface eth0)
25/03/07 20:10:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 20:10:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import os

In [3]:
path_arquivo = f'/home/jan/Documentos/tcc/datasets/prata/CARGA_ENERGIA/CARGA_ENERGIA_consolidada.parquet'
df = spark.read.parquet(path_arquivo)

In [4]:
df.show()

+-------------+--------------------+-------------------+---------------------+
|id_subsistema|      nom_subsistema|       din_instante|val_cargaenergiamwmed|
+-------------+--------------------+-------------------+---------------------+
|            N|               Norte|2013-12-31 22:00:00|        4629.20687500|
|           NE|            Nordeste|2013-12-31 22:00:00|        8104.89916667|
|            S|                 Sul|2013-12-31 22:00:00|        8451.91411621|
|           SE|Sudeste/Centro-Oeste|2013-12-31 22:00:00|       29770.29600000|
|            N|               Norte|2014-01-01 22:00:00|        5204.65166667|
|           NE|            Nordeste|2014-01-01 22:00:00|        9428.02975000|
|            S|                 Sul|2014-01-01 22:00:00|       10377.18163413|
|           SE|Sudeste/Centro-Oeste|2014-01-01 22:00:00|       36565.28716667|
|            N|               Norte|2014-01-02 22:00:00|        5394.63495833|
|           NE|            Nordeste|2014-01-02 22:00

In [5]:
df.createOrReplaceTempView("tb_carga_energia")

In [20]:
df_tratado = spark.sql("""
            SELECT 
                din_instante as dt_referencia,
                id_subsistema as sigla_regiao,
                nom_subsistema as nm_regiao,
                CAST(val_cargaenergiamwmed as DECIMAL(10,2)) as vl_cargaEnergiaMWmed
            FROM tb_carga_energia
            WHERE nom_subsistema like '%Sudeste%' AND val_cargaenergiamwmed <> '0E-8' 
                    """)

In [21]:
path_gold = '/home/jan/Documentos/tcc/datasets/gold/'

os.makedirs(path_gold, exist_ok=True)

In [22]:
df_tratado.toPandas().to_csv(f'{path_gold}/CARGA_ENERGIA.csv')

In [23]:
spark.stop()